# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in bham

## 1. Scraping postcode data from wikipedia

### I will use the pandas read_html function to build the dataframe

In [82]:
import pandas as pd


dfs = pd.read_csv('/home/nosamaj/Downloads/birmingham codes to ward.csv')

print(len(dfs))


77


In [83]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import numpy as np


### "dfs[0]" contains theh values we want. I will assume that all unasigned postcodes are marked 'Not assigned'


In [84]:

df = dfs
df =df.loc[df['ward'] != 'Not assigned']
df = df.reset_index(drop=True)
df.head()

,ward,postcode
0,Ladywood,B1
1,Bordesley Green,B10
2,Springfield,B11
3,Sparkbrook,B12
4,Moseley and Kings Heath,B13


In [85]:
df.shape

(77, 2)

## 2. Now we will geocode the postcode data using a supplied CSV file 

In [86]:
df_codes = pd.read_csv('/home/nosamaj/Downloads/postcode-outcodes.csv')


In [87]:
df_codes.head()

,id,postcode,latitude,longitude
0,2,AB10,57.13514,-2.11731
1,3,AB11,57.13875,-2.09089
2,4,AB12,57.10100,-2.11060
3,5,AB13,57.10801,-2.23776
4,6,AB14,57.10076,-2.27073


In [88]:
df =df.merge(df_codes, on='postcode')
df.head()

,ward,postcode,id,latitude,longitude
0,Bordesley Green,B10,46,52.46967,-1.85130
1,Springfield,B11,47,52.45440,-1.85871
2,Sparkbrook,B12,48,52.45982,-1.88225
3,Moseley and Kings Heath,B13,49,52.43644,-1.87790
4,Brandwood,B14,50,52.41757,-1.88824


## 3. We will now use the Forsquare API to cluster the postcode areas on ammenities

In [89]:
# get bham coordinate to centre map
from geopy.geocoders import Nominatim 

address = 'Birmingham, GB'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Birmingham are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Birmingham are 52.4796992, -1.9026911.


In [90]:
import folium 
map_birmingham = folium.Map(location=[latitude, longitude], zoom_start=11)

 #add markers to map
for lat, lng, ward, postcode in zip(df['latitude'], df['longitude'], df['ward'], df['postcode']):
    label = '{}, {}'.format(ward, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_birmingham)  
    
map_birmingham

### define credentials and version

In [91]:
CLIENT_ID = 'FMUYEQXGDOLYCP3A4DKAXAPO3J2VCHZ2CEQSA5IC1Z4103RO' # your Foursquare ID
CLIENT_SECRET = 'JUIE2JAQCJ3A13FGPFNANODGV22CTXFJSIC4X3ARPS5FFEWF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FMUYEQXGDOLYCP3A4DKAXAPO3J2VCHZ2CEQSA5IC1Z4103RO
CLIENT_SECRET:JUIE2JAQCJ3A13FGPFNANODGV22CTXFJSIC4X3ARPS5FFEWF


In [92]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ward', 
                  'Ward Latitude', 
                  'Ward Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [93]:
bham_venues = getNearbyVenues(names=df['ward'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'])

Bordesley Green
Springfield
Sparkbrook
Moseley and Kings Heath
Brandwood
Edgbaston
Ladywood
Harborne
Soho
Aston
Handsworth Wood
Soho
Stockland Green
Tyburn
South Yardley
Sheldon
Acocks Green
Hall Green
Selly Oak
Bournville
Northfield
Quinton
Stechford and Yardley North
Shard End
Tyburn
Castle Bromwich
Kingshurst and Fordbridge
Kings Norton
Bickenhill
Perry Barr
Newton
Oscott
Longbridge
Coleshill South
Drakes Cross and Walkers Heath
Alvechurch
Alcester
Bidford and Salford
Slideslow
Woodvale
Halesowen North
Belle Vale
Cradley Heath and Old Hill
Blackheath
Soho and Victoria
Smethwick
Old Warley
Oldbury
Greets Green and Lyng
Hateley Heath
Sutton New Hall
Sutton Vesey
Sutton Four Oaks
Sutton Trinity
Sutton New Hall
Wilnecote
Fazeley
Spital
Studley
Blythe
St Alphege
Elmdon
Knowle
Tanworth
Henley
Astwood Bank and Feckenham
Batchley & Brockhill
Greenlands


In [94]:
print(bham_venues.shape)
bham_venues.head()

(329, 7)


,ward,Ward Latitude,Ward Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bordesley Green,52.46967,-1.8513,Eis Café,52.469441,-1.857614,Café
1,Bordesley Green,52.46967,-1.8513,Subway,52.467907,-1.852995,Sandwich Place
2,Bordesley Green,52.46967,-1.8513,Hajees,52.468101,-1.853849,Asian Restaurant
3,Bordesley Green,52.46967,-1.8513,Simple Yet Creative,52.472774,-1.849508,Advertising Agency
4,Bordesley Green,52.46967,-1.8513,Alaturka,52.466680,-1.848568,Turkish Restaurant


In [95]:
bham_onehot = pd.get_dummies(bham_venues[['Venue Category']], prefix="", prefix_sep="")

# add ward column back to dataframe
bham_onehot['ward'] = bham_venues['ward'] 

# move ward column to the first column
fixed_columns = [bham_onehot.columns[-1]] + list(bham_onehot.columns[:-1])
bham_onehot = bham_onehot[fixed_columns]

bham_onehot.head()

,ward,Advertising Agency,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Supermarket,Tea Room,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Video Game Store,Warehouse Store,Wine Shop,Women's Store
0,Bordesley Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bordesley Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bordesley Green,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bordesley Green,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bordesley Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [96]:
#group the data

bham_grouped = bham_onehot.groupby('ward').mean().reset_index()
bham_grouped

,ward,Advertising Agency,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Supermarket,Tea Room,Toy / Game Store,Track Stadium,Train Station,Turkish Restaurant,Video Game Store,Warehouse Store,Wine Shop,Women's Store
0,Acocks Green,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.272727,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.0
1,Alcester,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.166667,0.166667,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,Aston,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,Batchley & Brockhill,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Belle Vale,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,Bickenhill,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,Bidford and Salford,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,Blackheath,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
8,Blythe,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0,0.000000,...,0.100000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,Bordesley Green,0.166667,0.000000,0.00,0.000000,0.0,0.166667,0.00,0.0,0.000000,...,0.000000,0.000000,0.000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.0


### find top 5 most common per neighbourhood

In [97]:
num_top_venues = 5

for hood in bham_grouped['ward']:
    print("----"+hood+"----")
    temp = bham_grouped[bham_grouped['ward'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acocks Green----
                    venue  freq
0             Supermarket  0.27
1                     Pub  0.09
2  Furniture / Home Store  0.09
3           Bowling Alley  0.09
4           Grocery Store  0.09


----Alcester----
               venue  freq
0  Indian Restaurant  0.17
1                Bar  0.17
2        Supermarket  0.17
3           Tea Room  0.17
4      Grocery Store  0.17


----Aston----
                venue  freq
0        Soccer Field  0.50
1  Photography Studio  0.25
2      Soccer Stadium  0.25
3  Advertising Agency  0.00
4       Movie Theater  0.00


----Batchley & Brockhill----
                        venue  freq
0  Construction & Landscaping  0.33
1                 Golf Course  0.33
2                        Park  0.33
3               Movie Theater  0.00
4                  Playground  0.00


----Belle Vale----
               venue  freq
0                Pub  0.25
1  Indian Restaurant  0.12
2        Pizza Place  0.12
3      Grocery Store  0.12
4               Par

In [98]:
### add these results to dataframe

First, let's write a function to sort the venues in descending order.

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create data frame

In [100]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ward']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_venues_sorted['ward'] = bham_grouped['ward']

for ind in np.arange(bham_grouped.shape[0]):
    Neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bham_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_venues_sorted.head()

,ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Acocks Green,Supermarket,Pub,Furniture / Home Store,Bowling Alley,Grocery Store
1,Alcester,Indian Restaurant,Bar,Supermarket,Tea Room,Grocery Store
2,Aston,Soccer Field,Photography Studio,Soccer Stadium,Advertising Agency,Movie Theater
3,Batchley & Brockhill,Construction & Landscaping,Golf Course,Park,Movie Theater,Playground
4,Belle Vale,Pub,Indian Restaurant,Pizza Place,Grocery Store,Park


### cluster the neightbourhoods

In [101]:
# set number of clusters
kclusters =10

bham_grouped_clustering = bham_grouped.drop('ward', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:234] 

array([1, 1, 3, 5, 2, 5, 6, 2, 5, 5, 2, 1, 5, 5, 0, 5, 9, 0, 5, 7, 5, 2,
       1, 5, 7, 0, 5, 5, 6, 5, 3, 1, 2, 7, 5, 1, 0, 6, 7, 3, 7, 1, 1, 5,
       2, 2, 5, 1, 5, 5, 5, 1, 1, 8, 3, 2, 0, 5, 4], dtype=int32)

In [102]:
# add clustering labels
Neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bham_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
bham_merged = bham_merged.join(Neighbourhoods_venues_sorted.set_index('ward'), on='ward')

bham_merged.head() # check the last columns!

,ward,postcode,id,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bordesley Green,B10,46,52.46967,-1.85130,5.0,Advertising Agency,Asian Restaurant,Turkish Restaurant,Gym / Fitness Center,Sandwich Place
1,Springfield,B11,47,52.45440,-1.85871,5.0,Sandwich Place,Park,Pakistani Restaurant,Advertising Agency,Pool
2,Sparkbrook,B12,48,52.45982,-1.88225,5.0,Indian Restaurant,Fast Food Restaurant,Grocery Store,Café,Pakistani Restaurant
3,Moseley and Kings Heath,B13,49,52.43644,-1.87790,NaN,NaN,NaN,NaN,NaN,NaN
4,Brandwood,B14,50,52.41757,-1.88824,1.0,Gym / Fitness Center,Restaurant,Chinese Restaurant,Grocery Store,Advertising Agency


In [103]:
bham_merged = bham_merged.fillna(0)
bham_merged.astype({'Cluster Labels': 'int32'}).dtypes


ward                      object
postcode                  object
id                         int64
latitude                 float64
longitude                float64
Cluster Labels             int32
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
dtype: object

In [104]:
#map the clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bham_merged['latitude'], bham_merged['longitude'], bham_merged['ward'], bham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The final map shows that there is a clear cluster of data around the south of the city centre of similar types. The famed "Balti Triange" of birmingham is mostly found withing the Sparkbrook Ward. This can be used as a benchmark to determine which classification contains the types of venues which are of interest. Based on the plot it can be seen that the cyan areas would make resonable targets for blockage investigations The Red category also appears spatially to be semi rural and the dark blue and purple to be located in suburban areas. This is expected and shows that the classification algorithm has produce feasible results. 